In [1]:
import os
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "madmex.settings")
import django
django.setup()

In [6]:
import madmex.models

In [2]:
from madmex.models import Object

In [5]:
Object.objects.all()[1]

(0.001) SELECT "madmex_object"."id", "madmex_object"."the_geom"::bytea, "madmex_object"."added" FROM "madmex_object" LIMIT 1 OFFSET 1; args=()


<Object: Object object (2)>

In [8]:
from madmex.settings import DATABASES

In [9]:
print(DATABASES)

{'default': {'ENGINE': 'django.contrib.gis.db.backends.postgis', 'USER': 'madmex_user', 'HOST': 'localhost', 'TIME_ZONE': None, 'OPTIONS': {}, 'TEST': {'NAME': None, 'CHARSET': None, 'MIRROR': None, 'COLLATION': None}, 'AUTOCOMMIT': True, 'CONN_MAX_AGE': 0, 'ATOMIC_REQUESTS': False, 'NAME': 'datacube_cluster', 'PASSWORD': 'qwerty', 'PORT': None}}


In [7]:
django.setup()